In [1]:

! pip install transformers gradio -q

     |████████████████████████████████| 5.8 MB 11.2 MB/s 
     |████████████████████████████████| 11.6 MB 23.7 MB/s 
     |████████████████████████████████| 7.6 MB 59.7 MB/s 
     |████████████████████████████████| 182 kB 76.7 MB/s 
     |████████████████████████████████| 84 kB 939 kB/s 
     |████████████████████████████████| 106 kB 81.6 MB/s 
     |████████████████████████████████| 2.3 MB 56.7 MB/s 
     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 54 kB 1.3 MB/s 
     |████████████████████████████████| 56 kB 2.6 MB/s 
     |████████████████████████████████| 213 kB 68.4 MB/s 
     |████████████████████████████████| 278 kB 74.6 MB/s 
     |████████████████████████████████| 84 kB 1.5 MB/s 
     |████████████████████████████████| 64 kB 3.5 MB/s 
     |████████████████████████████████| 80 kB 11.1 MB/s 
     |████████████████████████████████| 68 kB 8.8 MB/s 
     |████████████████████████████████| 68 kB 8.1 MB/s 
     |████████████████████████

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import gradio as gr

In [3]:
# Instantiate model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [4]:
def predict(input, history=[]):

    # Basic instruction
    instruction = 'Instruction: given a dialog context, you need to response empathically.'

    # Knowledge given
    knowledge = '  '

    # Copy history of dialog
    dialog_list = history.copy()
    print('history of dialog: ', dialog_list)

    # Append user input to dialog
    dialog_list.append(input)
    print('new input appended: ', dialog_list)

    # Prepare dialog to be used in the model
    dialog = ' EOS '.join(dialog_list)
    print('dialog prepared for the model: ', dialog)

    # Build query 
    # Question: What does [CONTEXT] do???
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
    print('query: ', query)

    # Tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(f"{query}", return_tensors='pt')
    print('Output of the tokenizer: ', new_user_input_ids)

    # Set params for model output
    top_p = 0.9 # What is this???
    min_length = 8
    max_length = 200

    # Generate output
    output = model.generate(new_user_input_ids, 
                            min_length=int(min_length), 
                            max_length=int(max_length), 
                            top_p=top_p, 
                            do_sample=True).tolist()
    print('Output of the model: ', output)          

    # Decode output
    response = tokenizer.decode(output[0], skip_special_tokens=True)   

    # Create new history of dialog
    dialog_list.append(response)

    return response, dialog_list

In [5]:
# First user input
input1 = 'Hi. I feel sad and I would love to get some help.'

In [6]:
# First response
response1, history1 = predict(input1)
print(response1)
print(history1)

history of dialog:  []
new input appended:  ['Hi. I feel sad and I would love to get some help.']
dialog prepared for the model:  Hi. I feel sad and I would love to get some help.
query:  Instruction: given a dialog context, you need to response empathically. [CONTEXT] Hi. I feel sad and I would love to get some help.   
Output of the tokenizer:  tensor([[21035,    10,   787,     3,     9, 13463,  2625,     6,    25,   174,
            12,  1773,  8943,     9,   189,  6402,     5,   784, 17752,  3463,
             4,   382,   908,  2018,     5,    27,   473,  6819,    11,    27,
           133,   333,    12,   129,   128,   199,     5,     1]])
Output of the model:  [[0, 2018, 6, 27, 31, 51, 6819, 24, 25, 31, 60, 352, 190, 48, 5, 1]]
Hi, I'm sad that you're going through this.
['Hi. I feel sad and I would love to get some help.', "Hi, I'm sad that you're going through this."]


In [7]:
# Second user input
input2 = 'Can you help me feeling better?'


In [8]:
# Second response
response2, history2 = predict(input2, history1)
print(response2)
print(history2)

history of dialog:  ['Hi. I feel sad and I would love to get some help.', "Hi, I'm sad that you're going through this."]
new input appended:  ['Hi. I feel sad and I would love to get some help.', "Hi, I'm sad that you're going through this.", 'Can you help me feeling better?']
dialog prepared for the model:  Hi. I feel sad and I would love to get some help. EOS Hi, I'm sad that you're going through this. EOS Can you help me feeling better?
query:  Instruction: given a dialog context, you need to response empathically. [CONTEXT] Hi. I feel sad and I would love to get some help. EOS Hi, I'm sad that you're going through this. EOS Can you help me feeling better?   
Output of the tokenizer:  tensor([[21035,    10,   787,     3,     9, 13463,  2625,     6,    25,   174,
            12,  1773,  8943,     9,   189,  6402,     5,   784, 17752,  3463,
             4,   382,   908,  2018,     5,    27,   473,  6819,    11,    27,
           133,   333,    12,   129,   128,   199,     5, 32100,  

In [9]:
# Third user input
input3 = 'That is cool! What should I do?'

In [10]:
# Third response
response3, history3 = predict(input3, history2)
print(response3)
print(history3)

history of dialog:  ['Hi. I feel sad and I would love to get some help.', "Hi, I'm sad that you're going through this.", 'Can you help me feeling better?', 'I can help you too, I am here to help you.']
new input appended:  ['Hi. I feel sad and I would love to get some help.', "Hi, I'm sad that you're going through this.", 'Can you help me feeling better?', 'I can help you too, I am here to help you.', 'That is cool! What should I do?']
dialog prepared for the model:  Hi. I feel sad and I would love to get some help. EOS Hi, I'm sad that you're going through this. EOS Can you help me feeling better? EOS I can help you too, I am here to help you. EOS That is cool! What should I do?
query:  Instruction: given a dialog context, you need to response empathically. [CONTEXT] Hi. I feel sad and I would love to get some help. EOS Hi, I'm sad that you're going through this. EOS Can you help me feeling better? EOS I can help you too, I am here to help you. EOS That is cool! What should I do?   
O